# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

## Preprocesamiento

Problemas a resolver en el preprocesamiento de los datos:
* Informacion desconocida (unknown, Unknown, NaN)
* Entradas categóricas nominales y ordinales

### Librerias utilizadas

In [ ]:
import pandas
import numpy
import sklearn
import warnings
import matplotlib.pyplot as plt
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from feature_engine.imputation import CategoricalImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from category_encoders import BinaryEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import plotly.offline as pyo
pyo.init_notebook_mode()
from sklearn.metrics import recall_score
import plotly.graph_objects as go
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import LearningCurve
from collections import defaultdict
import seaborn as sns



warnings.filterwarnings('ignore')

In [ ]:
accidentes = pandas.read_csv('accidentes.csv')
accidentes.head()

In [ ]:
## Distribución de nulos
accidentes.replace(["Unknown","unknown"], numpy.NaN, inplace=True)
accidentes.isnull().sum()

Las entradas que tengan menos de 500 nulos se ignorarán ya que se tiene suficientes datos menos sexo que es facil de inputarle valor.

In [ ]:
accidentes.dropna(subset=
        ['medians',
         'surface',
         'weather',
         'collision',
         'car_movement',
         'acc_cause'],
        inplace=True)

accidentes.isnull().sum()

In [ ]:
# División del conjunto de datos: 60% train, 20% test, 20% validation

train, not_train = train_test_split(accidentes, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

Se utilizará un **DataFrameMapper** para dejar los datos "listos". En este caso se imputarán los valores **NaN** con un **CategoricalImputer** en donde falten, este imputer es utilizado justamente para variables categóricas. Además, las variables ordinales como **'age'** y **'sex'** se codificarán con **OrdinalEncoder** y las variables nominales con  **OneHotEncoder**

In [ ]:
age_encoder = OrdinalEncoder()
sex_encoder = OrdinalEncoder()

mapeador1 = DataFrameMapper( 
            [
                (['age'],[CategoricalImputer(imputation_method='frequent'), age_encoder]),
                (['sex'],[CategoricalImputer(imputation_method='frequent'), sex_encoder]),
                (['exp'],[CategoricalImputer(imputation_method='frequent'), OneHotEncoder()]),
                (['medians'],[OneHotEncoder()]),
                (['junction'],[CategoricalImputer(imputation_method='frequent'), OneHotEncoder()]),
                (['surface'],[OneHotEncoder()]),
                (['light'],[OneHotEncoder()]),
                (['weather'],[OneHotEncoder()]),
                (['collision'],[OneHotEncoder()]),
                (['car_movement'],[OneHotEncoder()]),
                (['acc_cause'],[OneHotEncoder()]),
            ])

mapeador1.fit(train)
muestra = train.sample(1)
mapeador1.transform(muestra)

In [ ]:
mapeador1.transformed_names_

In [ ]:
## Cuales son las categorias de ordinales?

age_encoder.categories_ ## ['18-30': 0, '31-50': 1, 'Over 51': 2, 'Under 18': 3]

In [ ]:
sex_encoder.categories_ ## ['Female' : 0, 'Male' : 1]

# Métrica

El sentido de utilizar un clasificador para la informacion que aquí se dispone de accidentes de tráfico es tratar de preever accidentes según su gravedad. Por lo tanto, la métrica **Acuracy** no interesa en estas instancias ya que ésta es indiferente a la gravedad del accidente. Más bien conviene la métrica **Recall** para que no se escape ningún caso de las clases, sobre todo si es fatal. Tampoco se observará tanto la métrica **Precision** ya que no es tan grave definir como fatal un accidente mientras no lo era.
Se observará también **Recall** con el promedio general ponderado de las tres clases con el parámetro **average='weighted'** para tener una medida general del Recall atentiendo al desbalanceo de la información.

# Ingeniería de variables de entrada

Debido a la gran cantidad de dimensiones generadas a partir de aplicar el codificador **One-Hot Encoder** a las variables de entradas, seria interesante lograr una considerable reducción de dimensionalidad utilizando algun otro codificador y que no sea tan costoso aprender para los modelos. Para esto, se eligió utilizar el codificador **BinaryEncoder** que primero codifica las variables categóricas en números discretos y luego se generan cantidad de columnas como digitos necesarios para representar esas categoricas en binario.

In [ ]:
age_encoder = OrdinalEncoder()
sex_encoder = OrdinalEncoder()


mapeador2 = DataFrameMapper( 
            [
                (['age'],[CategoricalImputer(imputation_method='frequent'), age_encoder]),
                (['sex'],[CategoricalImputer(imputation_method='frequent'), sex_encoder]),
                (['exp'],[CategoricalImputer(imputation_method='frequent'), OneHotEncoder()]),
                (['medians'],[BinaryEncoder()]),
                (['junction'],[CategoricalImputer(imputation_method='frequent'), BinaryEncoder()]),
                (['surface'],[BinaryEncoder()]),
                (['light'],[BinaryEncoder()]),
                (['weather'],[BinaryEncoder()]),
                (['collision'],[BinaryEncoder()]),
                (['car_movement'],[BinaryEncoder()]),
                (['acc_cause'],[BinaryEncoder()]),
            ])

mapeador2.fit(train)

muestra = train.sample(1)

mapeador2.transform(muestra)

In [ ]:
mapeador2.transformed_names_

Se observa una considerable reducción en la dimensionalidad contra el uso de **OHE**. La desventeja de aplicar esta ingenieria en las variables de entradas es que se puede perder un poco de transparencia para el cliente.


A continuación se evalua la métrica seleccionada y la ingenieria de entrada con los tres modelos seleccionados sin mejora de hiperparametros

# K vecinos más cercanos
A continuacion se entrena **KNN** con **train** y se evalua la métrica **Recall** sin aplicar **BinaryEncoder**

In [ ]:
tuberia_knn = Pipeline([
    ('mapper', mapeador1),
    ('classifier', KNeighborsClassifier(n_neighbors=2)),
])

tuberia_knn.fit(train, train.severity)
prediccion_knn = tuberia_knn.predict(train)

df = pandas.DataFrame(prediccion_knn, columns=['severity'])
df.groupby('severity').aggregate({'severity':'count'})

In [ ]:
print(metrics.classification_report(train.severity, prediccion_knn))

A continuacion se entrena **KNN** con **train** y se evalua la métrica **Recall** aplicando **BinaryEncoder**

In [ ]:
tuberia_knn_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', KNeighborsClassifier(n_neighbors=2)),
])

tuberia_knn_2.fit(train, train.severity)
prediccion_knn_2 = tuberia_knn_2.predict(train)

df_2 = pandas.DataFrame(prediccion_knn_2, columns=['severity'])
df_2.groupby('severity').aggregate({'severity':'count'})

In [ ]:
print(metrics.classification_report(train.severity, prediccion_knn_2))

# Árboles de decisión

A continuacion se entrena **Tree** con **train** y se evalua la métrica **Recall** sin aplicar **BinaryEncoder**

In [ ]:
tuberia_arbol = Pipeline([
    ('mapper', mapeador1),
    ('classifier', DecisionTreeClassifier(max_depth=10, class_weight='balanced')),
])

tuberia_arbol.fit(train, train.severity)
prediccion_arbol = tuberia_arbol.predict(train)

prediccion_arbol

df_arbol = pandas.DataFrame(prediccion_arbol, columns=['severity'])
df_arbol.groupby('severity').aggregate({'severity':'count'})

In [ ]:
print(metrics.classification_report(train.severity, prediccion_arbol))

A continuacion se entrena **Tree** con **train** y se evalua la métrica **Recall** aplicando **BinaryEncoder**

In [ ]:
tuberia_arbol_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', DecisionTreeClassifier(max_depth=10, class_weight='balanced')),
])

tuberia_arbol_2.fit(train, train.severity)
prediccion_arbol_2 = tuberia_arbol_2.predict(train)

df_arbol_2 = pandas.DataFrame(prediccion_arbol_2, columns=['severity'])
df_arbol_2.groupby('severity').aggregate({'severity':'count'})

In [ ]:
print(metrics.classification_report(train.severity, prediccion_arbol_2))

# Árboles potenciados por gradiente

A continuacion se entrena **GBOO** con **train** y se evalua la métrica **Recall** sin aplicar **BinaryEncoder**

In [ ]:
tuberia_boo = Pipeline([
    ('mapper', mapeador1),
    ('classifier', GradientBoostingClassifier(max_depth=10)),
])

tuberia_boo.fit(train, train.severity)
prediccion_boo = tuberia_boo.predict(train)

df_arbol = pandas.DataFrame(prediccion_boo, columns=['severity'])
df_arbol.groupby('severity').aggregate({'severity':'count'})

In [ ]:
print(metrics.classification_report(train.severity, prediccion_boo))

A continuacion se entrena **GBOO** con **train** y se evalua la métrica **Recall** aplicando **BinaryEncoder**

In [ ]:
tuberia_boo_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', GradientBoostingClassifier(max_depth=10)),
])

tuberia_boo_2.fit(train, train.severity)
prediccion_boo_2 = tuberia_boo_2.predict(train)

df_arbol = pandas.DataFrame(prediccion_boo_2, columns=['severity'])
df_arbol.groupby('severity').aggregate({'severity':'count'})

In [ ]:
print(metrics.classification_report(train.severity, prediccion_boo_2))

Efectivamente los modelos funcionanan mejorar aplicando ingenieria de entradas con **BinaryEncoder** a las entradas categóricas

# Evaluación de modelos

In [ ]:
## Definicion de funcion para evaluar el modelo
def evaluate_model(model, set_names=('train', 'validation'), title='', show_cm=True):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)
    
    if show_cm:
        fig, axis = plt.subplots(1, len(set_names), sharey=True, figsize=(15, 3))
    
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name]  # <- hack feo...

        y = set_data.severity
        y_pred = model.predict(set_data)
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred, average='weighted'))
        
        if show_cm:
            ax = axis[i]
            sns.heatmap(metrics.confusion_matrix(y, y_pred), ax=ax, cmap='Blues', annot=True, fmt='.0f', cbar=False)

            ax.set_title(set_name)
            ax.xaxis.set_ticklabels(['fatal', 'grave', 'leve'])
            ax.yaxis.set_ticklabels(['fatal', 'grave', 'leve'])
            ax.set_xlabel('Predicted class')
            ax.set_ylabel('True class')

        
    display(pandas.DataFrame(final_metrics, index=set_names))
    if show_cm:
        plt.tight_layout()
        plt.show()

### Hiperparámetros a exporar con KNN

* Cantidad de vecinos K
* Medicion de distancia: euclidean, manhattan, minkowski

Se procede a buscar estocasticamente un valor de **K** óptimo teniendo en cuenta la métrica seleccionada.

In [ ]:
k_top = 20
results = numpy.zeros((k_top, 2))

X, y = train, train.severity

for i in range(1, k_top):
    k_values_train = []
    k_values_test = []
    for t in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

        model = KNeighborsClassifier(n_neighbors=i)
        
        tuberia_knn_2 = Pipeline([
            ('mapper', mapeador2),
            ('classifier', model),
        ])

        tuberia_knn_2.fit(X_train, y_train)
        prediccion_knn_2_train = tuberia_knn_2.predict(X_train)
        prediccion_knn_2_test = tuberia_knn_2.predict(X_test)

        k_values_train.append(metrics.recall_score(y_train, prediccion_knn_2_train, average='weighted'))
        k_values_test.append(metrics.recall_score(y_test, prediccion_knn_2_test,average='weighted'))
    results[i-1, 0] = numpy.mean(k_values_train)
    results[i-1, 1] = numpy.mean(k_values_test)

In [ ]:
fig = go.Figure()
x_values = list(range(1, k_top))
fig.add_trace(go.Scatter(x=x_values, y=results[:, 0],
                    mode='lines',
                    name='train'))
fig.add_trace(go.Scatter(x=x_values, y=results[:, 1],
                    mode='lines',
                    name='test'))
fig.show()

Se nota (como es conocido) que **KNN** es altamente inestable al principio en un rango de K=(1-8) luego se estabiliza con valores del **promedio de Recall ponderado** de hasta el 84%. De ahora en adelante se utilizará el valor 10 para **K**.

A continuación se evalua **KNN** con distintas maneras de medir la distancias:

In [ ]:
## Midiendo distancias euclídeas
tuberia_knn_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', KNeighborsClassifier(n_neighbors=10, weights='distance', metric='euclidean'))
])

tuberia_knn_2.fit(train, train.severity)
prediccion_knn_2 = tuberia_knn_2.predict(train)
    
metrics.recall_score(train.severity, prediccion_knn_2, average='weighted')

In [ ]:
## Midiendo distancias manhattan
tuberia_knn_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', KNeighborsClassifier(n_neighbors=10, weights='distance', metric='manhattan'))
])

tuberia_knn_2.fit(train, train.severity)
prediccion_knn_2 = tuberia_knn_2.predict(train)
    
evaluate_model(tuberia_knn_2,set_names=('train', 'validation','test') ,title='', show_cm=True )

Es notable una gran mejoría de mas de 10 puntos porcentuales en la métrica de promedio ponderado de **Recall** utilizando otras maneras de medir las distancias distintas a **minkowski** como **euclidean** o **manhatan**. El modelo efectivamente funciona mejor con estos parámetros.

### Hiperparámetros a explorar con Árboles de decisión
* Profundidad
* Ponderacion de las clases
* Criterio de separación

In [ ]:
## Máxima profundidad 5 sin balanceo de clases
tuberia_arbol_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', DecisionTreeClassifier(max_depth=5)),
])

tuberia_arbol_2.fit(train, train.severity)
prediccion_arbol_2 = tuberia_arbol_2.predict(train)

print(metrics.classification_report(train.severity, prediccion_arbol_2))

In [ ]:
## Máxima profundidad 10 sin balanceo de clases
tuberia_arbol_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', DecisionTreeClassifier(max_depth=10)),
])

tuberia_arbol_2.fit(train, train.severity)
prediccion_arbol_2 = tuberia_arbol_2.predict(train)

print(metrics.classification_report(train.severity, prediccion_arbol_2))

Se observa que solo clasifica bien para la clase mayoritaria, el problema de el desbalanceo de las clases es evidente

In [ ]:
## Máxima profundidad 15 con balanceo de clases
tuberia_arbol_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', DecisionTreeClassifier(max_depth=15, class_weight='balanced')),
])

tuberia_arbol_2.fit(train, train.severity)
prediccion_arbol_2 = tuberia_arbol_2.predict(train)

evaluate_model(tuberia_arbol_2,set_names=('train', 'validation','test') ,title='', show_cm=True )

In [ ]:
## Máxima profundidad 10 con balanceo de clases y entropía como criterio de división
tuberia_arbol_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', DecisionTreeClassifier(max_depth=10, class_weight='balanced', criterion='entropy'),
])

tuberia_arbol_2.fit(train, train.severity)
prediccion_arbol_2 = tuberia_arbol_2.predict(train)

print(metrics.classification_report(train.severity, prediccion_arbol_2))

Gracias al balanceo de datos, se observa una gran mejora en la clasificaciones de clases **Fatales** que son las que mas interesan, pero tanto, que parece que sobreentrena en esos casos. El criterio **'gini'** parece ser mejor que el entrópico

### Hiperparámetros a explorar con Árboles de decisión potencias por gradiente
* Profundidad
* Cantidad minimas de instancias por separacion
* Clasificadores

In [ ]:
## Maxima profundidad de 10 y clasificadores 50
tuberia_boo_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', GradientBoostingClassifier(max_depth=10,
                                              n_estimators=50
                                             )),
])

tuberia_boo_2.fit(train, train.severity)
prediccion_boo_2 = tuberia_boo_2.predict(train)

print(metrics.classification_report(train.severity, prediccion_boo_2))

In [ ]:
## Maxima profundidad de 10, clasificadores 50 y minima cantidad de 80 instancias para separacion
tuberia_boo_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', GradientBoostingClassifier(max_depth=10,
                                              n_estimators=50,
                                              min_samples_split=80,
                                             )),
])

tuberia_boo_2.fit(train, train.severity)
prediccion_boo_2 = tuberia_boo_2.predict(train)

print(metrics.classification_report(train.severity, prediccion_boo_2))

In [ ]:
## Maxima profundidad de 10, clasificadores 50 y minima cantidad de 10 instancias para separacion
tuberia_boo_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', GradientBoostingClassifier(max_depth=10,
                                              n_estimators=50,
                                              min_samples_split=10,
                                             )),
])

tuberia_boo_2.fit(train, train.severity)
prediccion_boo_2 = tuberia_boo_2.predict(train)

print(metrics.classification_report(train.severity, prediccion_boo_2))

### Elección de modelo

Lo que se observa con **arboles potenciados por gradiente** es en aparaciencia el modelo mas potente de los tres para clasificación **multiclases desbalanceadas**, pero tanto que parece **sobreentrenar**, la calidad de la clasificación cae cuando subimos el número de instancias para cada separación. Se podria utilizar este y otros parametros para controlar el sobreentrenamiento.
Por estas razones, se decidió utilizar este modelo como el pricipal. Se procede a explotar los parámetros aún mas y controlar el sobreentrenamiento

In [ ]:
## Maxima profundidad de 10, clasificadores 50 y minima cantidad de 10 instancias para separacion
tuberia_boo_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', GradientBoostingClassifier(max_depth=10,
                                              n_estimators=50,
                                              min_samples_split=10,
                                             )),
])

In [ ]:
cv = StratifiedKFold(n_splits=12)
tamanios = np.linspace(0.3, 1.0, 10)

visualizador = LearningCurve(
    tuberia_boo_2, cv=cv, scoring='recall_weighted', train_sizes=tamanios, n_jobs=-1
)

visualizador.fit(train, train.severity) 
visualizador.show()          

Con un modelo de arboles de decisión potenciados por grandes con **maxima profundidad de 10, clasificadores 50 y minima cantidad de 10 instancias para separacion** observamos un sobre-entrenamiento. A pesar de esto, el gráfico parece indicar que si continuamos entrendo contra mas KFolds el aprendizaje iría aumentando.

A continuacion probamos otro modelo que minimice el sobre-entrenamiento:

In [ ]:
tuberia_boo_2 = Pipeline([
    ('mapper', mapeador2),
    ('classifier', GradientBoostingClassifier(max_depth=8,
                                              n_estimators=150,
                                              min_samples_split=40,
                                              min_samples_leaf=8
                                             )),
])

tuberia_boo_2.fit(train, train.severity)
prediccion_boo_2 = tuberia_boo_2.predict(train)

print(metrics.classification_report(train.severity, prediccion_boo_2))

In [ ]:
evaluate_model(tuberia_boo_2,set_names=('train', 'validation','test') ,title='', show_cm=True )

### Conclusión
A pesar del sobreentrenamiento, de los tres modelos, el mejor indica a ser el arbol potenciado por gradiente eso se muestra con un recall promedio ponderado en test del 83%. Sin embargo, sigue siendo muy dificil clasificar correctamente las clases minoritarias, ya que estan MUY desbalanceadas y son tan pocos los casos de fatales que podrian hasta ser considerados outliers. Podrian mejorarse estas situaciones con un muy fino balanceo de arboles buscando los parametros con GridSearchCV y quizas de alguna manera no borrar tantas filas para tener mas datos.